In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
PATH = "/content/drive/MyDrive/kaggle_competition_data/"

In [3]:
import pandas as pd

matches_df = pd.read_csv(f"{PATH}/data/train_matches.csv").head(500000)
# matches_df = pd.read_csv(f"{PATH}/data/train_matches.csv")
user_df = pd.read_csv(f"{PATH}/data/train_users.csv").set_index("user_id")
reviews_df = pd.read_csv(f"{PATH}/data/train_reviews.csv").set_index("review_id")

In [4]:
users_df = user_df.loc[matches_df['user_id']]
reviews_df = reviews_df.loc[matches_df['review_id']]

In [5]:
users_df = users_df.reset_index()
reviews_df = reviews_df.reset_index()

In [6]:
len(users_df)

500000

In [7]:
import torch

# Check if GPU is available and set the device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [8]:
device

device(type='cuda')

In [9]:
import warnings
warnings.filterwarnings('ignore')
import numpy as np
import pandas as pd
from transformers import AutoTokenizer

# Define Encoder Module
class Encoder(torch.nn.Module):
    def __init__(self, vocab_size, embed_dim, output_embed_dim):
        super().__init__()
        self.embedding_layer = torch.nn.Embedding(vocab_size, embed_dim)
        self.encoder = torch.nn.TransformerEncoder(
            torch.nn.TransformerEncoderLayer(embed_dim, nhead=8, batch_first=True),
            num_layers=3,
            norm=torch.nn.LayerNorm([embed_dim]),
            enable_nested_tensor=False
        )
        self.projection = torch.nn.Linear(embed_dim, output_embed_dim)

    def forward(self, tokenizer_output):
        x = self.embedding_layer(tokenizer_output['input_ids'])
        x = self.encoder(x, src_key_padding_mask=tokenizer_output['attention_mask'].logical_not())
        cls_embed = x[:, 0, :]
        return self.projection(cls_embed)


In [10]:
import calendar

#Here we modify the column location
def transform_location(row):
    location = ""
    if row["location_is_ski"] == 1:
        location = "in a ski resort"
    elif row["location_is_beach"] == 1:
        location = "near a beach"
    elif row["location_is_city_center"] == 1:
        location = "in a city center"
    else:
        location = "not mentionned"
    return location

def get_review_text_representation(row):
  review = ""
  if row['review_title']:
      review += f"review_title: {row['review_title']} / "
  if row['review_positive']:
      review = f"positive review: {row['review_positive']} / "
  if row['review_negative']:
      review += f"negative review: {row['review_negative']}"
  if review.count(" ") < 30:
    return None
  text = f"""The guest gave a score of {row['review_score']}/10 to the accommodation, Here is what the guest thought about the accommodation during his stay: {review}. The guest's review has been helpful for {row['review_helpful_votes']} other guests."""
  return text

def get_accommodation_text_representation(row):
  month = calendar.month_name[row["month"]]
  text = f"""The guest was a {row['guest_type']} coming from {row['guest_country']}, he booked a {row['accommodation_type']} in {row['accommodation_country']} for {row['room_nights']} nights during the month of {month}.
  The accommodation he booked had an everage grade of {row['accommodation_score']}/10 and a star rating of {row['accommodation_star_rating']} stars. The location of the accommodation was {row['location']}.
  """
  return text

In [11]:
merged_df = matches_df.merge(user_df, on=['user_id', 'accommodation_id'])
merged_df = merged_df.merge(reviews_df, on='review_id')
merged_df['review_text'] = merged_df.apply(get_review_text_representation, axis=1)
merged_df = merged_df.dropna(subset=['review_text'])
len(merged_df)

305144

In [12]:
merged_df['location'] = merged_df.apply(transform_location, axis=1)
merged_df['user_accommodation_info'] = merged_df.apply(get_accommodation_text_representation, axis=1)

In [14]:
merged_df[['user_accommodation_info', 'review_text']]

,user_accommodation_info,review_text
0,"The guest was a Couple coming from Zot, he boo...",The guest gave a score of 8.0/10 to the accomm...
1,"The guest was a Group coming from Zuc, he book...",The guest gave a score of 10.0/10 to the accom...
2,The guest was a Family with children coming fr...,The guest gave a score of 8.0/10 to the accomm...
3,"The guest was a Couple coming from Zuc, he boo...",The guest gave a score of 9.0/10 to the accomm...
5,"The guest was a Group coming from Rawabe, he b...",The guest gave a score of 9.0/10 to the accomm...
...,...,...
499994,"The guest was a Group coming from Zuc, he book...",The guest gave a score of 7.0/10 to the accomm...
499995,"The guest was a Couple coming from Wapef, he b...",The guest gave a score of 10.0/10 to the accom...
499996,"The guest was a Couple coming from Mejok, he b...",The guest gave a score of 9.0/10 to the accomm...
499997,The guest was a Family with children coming fr...,The guest gave a score of 10.0/10 to the accom...


In [13]:
merged_df[["review_text"]]

,review_text
0,The guest gave a score of 8.0/10 to the accomm...
1,The guest gave a score of 10.0/10 to the accom...
2,The guest gave a score of 8.0/10 to the accomm...
3,The guest gave a score of 9.0/10 to the accomm...
5,The guest gave a score of 9.0/10 to the accomm...
...,...
499994,The guest gave a score of 7.0/10 to the accomm...
499995,The guest gave a score of 10.0/10 to the accom...
499996,The guest gave a score of 9.0/10 to the accomm...
499997,The guest gave a score of 10.0/10 to the accom...


In [15]:
# Custom Dataset Class
class BookingDataset(torch.utils.data.Dataset):
    def __init__(self, merged_df):
        self.data = merged_df[['user_accommodation_info', 'review_text']]

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        user_accommodation = self.data.iloc[idx]['user_accommodation_info']
        review = self.data.iloc[idx]['review_text']
        return user_accommodation, review


TRAINING THE MODEL

In [16]:
# Training Loop
def train(dataset, num_epochs=10):
    embed_size = 512
    output_embed_size = 128
    max_seq_len = 64
    batch_size = 1000

    tokenizer = AutoTokenizer.from_pretrained('bert-base-uncased')
    user_acc_encoder = Encoder(tokenizer.vocab_size, embed_size, output_embed_size).to(device)
    review_encoder = Encoder(tokenizer.vocab_size, embed_size, output_embed_size).to(device)

    dataloader = torch.utils.data.DataLoader(dataset, batch_size=batch_size, shuffle=True)
    optimizer = torch.optim.Adam(
        list(user_acc_encoder.parameters()) + list(review_encoder.parameters()), lr=0.0001
    )
    loss_fn = torch.nn.CrossEntropyLoss()

    for epoch in range(num_epochs):
        running_loss = []
        for idx, data_batch in enumerate(dataloader):
            user_acc, review = data_batch
            user_acc_tok = tokenizer(user_acc, padding=True, truncation=True, return_tensors='pt', max_length=max_seq_len)
            review_tok = tokenizer(review, padding=True, truncation=True, return_tensors='pt', max_length=max_seq_len)

            # Move tokenized inputs to the GPU
            user_acc_tok = {key: val.to(device) for key, val in user_acc_tok.items()}
            review_tok = {key: val.to(device) for key, val in review_tok.items()}

            user_acc_embed = user_acc_encoder(user_acc_tok)
            review_embed = review_encoder(review_tok)

            similarity_scores = user_acc_embed @ review_embed.T
            target = torch.arange(user_acc_embed.shape[0], dtype=torch.long, device=device)
            loss = loss_fn(similarity_scores, target)
            running_loss.append(loss.item())

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

        print(f"Epoch {epoch}, loss = {np.mean(running_loss)}")

    return user_acc_encoder, review_encoder

In [17]:
# Load data and train model
dataset = BookingDataset(merged_df)
user_acc_encoder, review_encoder = train(dataset, num_epochs=80)

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

Epoch 0, loss = 6.2600493586920445
Epoch 1, loss = 5.620654513633329
Epoch 2, loss = 5.349453608974133
Epoch 3, loss = 5.174327490376491
Epoch 4, loss = 5.034029714422288
Epoch 5, loss = 4.92082591539894
Epoch 6, loss = 4.826184817388946
Epoch 7, loss = 4.7362178205664645
Epoch 8, loss = 4.651015450751859
Epoch 9, loss = 4.574037519155764
Epoch 10, loss = 4.500047834870083
Epoch 11, loss = 4.428660642087849
Epoch 12, loss = 4.358374071277045
Epoch 13, loss = 4.286169618562935
Epoch 14, loss = 4.21994098962522
Epoch 15, loss = 4.1518783491421365
Epoch 16, loss = 4.084062707969566
Epoch 17, loss = 4.015355750626209
Epoch 18, loss = 3.9514211237041
Epoch 19, loss = 3.879086866098292
Epoch 20, loss = 3.813743485344781
Epoch 21, loss = 3.749511842634164
Epoch 22, loss = 3.6811965127396427
Epoch 23, loss = 3.610302256602867
Epoch 24, loss = 3.544027055011076
Epoch 25, loss = 3.4783122407065497
Epoch 26, loss = 3.410032621785706
Epoch 27, loss = 3.344641709249783
Epoch 28, loss = 3.2777350019

In [18]:
import pickle

# Save the encoder models using pickle
with open(f"{PATH}/user_acc_encoder.pkl", 'wb') as f:
    pickle.dump(user_acc_encoder, f)

with open(f"{PATH}/review_encoder.pkl", 'wb') as f:
    pickle.dump(review_encoder, f)


TESTING THE MODEL

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
PATH = "/content/drive/MyDrive/kaggle_competition_data/"

In [3]:
import warnings
import torch
warnings.filterwarnings('ignore')
import numpy as np
import pandas as pd
from transformers import AutoTokenizer

# Define Encoder Module
class Encoder(torch.nn.Module):
    def __init__(self, vocab_size, embed_dim, output_embed_dim):
        super().__init__()
        self.embedding_layer = torch.nn.Embedding(vocab_size, embed_dim)
        self.encoder = torch.nn.TransformerEncoder(
            torch.nn.TransformerEncoderLayer(embed_dim, nhead=8, batch_first=True),
            num_layers=3,
            norm=torch.nn.LayerNorm([embed_dim]),
            enable_nested_tensor=False
        )
        self.projection = torch.nn.Linear(embed_dim, output_embed_dim)

    def forward(self, tokenizer_output):
        x = self.embedding_layer(tokenizer_output['input_ids'])
        x = self.encoder(x, src_key_padding_mask=tokenizer_output['attention_mask'].logical_not())
        cls_embed = x[:, 0, :]
        return self.projection(cls_embed)


In [4]:
import pickle

# Load the encoder models using pickle
with open(f"{PATH}/user_acc_encoder.pkl", 'rb') as f:
    user_acc_encoder = pickle.load(f)

with open(f"{PATH}/review_encoder.pkl", 'rb') as f:
    review_encoder = pickle.load(f)

In [5]:
import pandas as pd

users_df = pd.read_csv(f"{PATH}/data/test_users.csv")
reviews_df = pd.read_csv(f"{PATH}/data/test_reviews.csv")

In [6]:
def get_review_text_representation_test(row):
  review = ""
  if row['review_title']:
      review += f"review_title: {row['review_title']} / "
  if row['review_positive']:
      review = f"positive review: {row['review_positive']} / "
  if row['review_negative']:
      review += f"negative review: {row['review_negative']}"
  text = f"""The guest gave a score of {row['review_score']}/10 to the accommodation, Here is what the guest thought about the accommodation during his stay: {review}. The guest's review has been helpful for {row['review_helpful_votes']} other guests."""
  return text

In [7]:
#Here we modify the column location
def transform_location(row):
    location = ""
    if row["location_is_ski"] == 1:
        location = "in a ski resort"
    elif row["location_is_beach"] == 1:
        location = "near a beach"
    elif row["location_is_city_center"] == 1:
        location = "in a city center"
    else:
        location = "not mentionned"
    return location

In [8]:
users_df['location'] = users_df.apply(transform_location, axis=1)

In [9]:
import calendar

def get_accommodation_text_representation(row):
  month = calendar.month_name[row["month"]]
  text = f"""The guest was a {row['guest_type']} coming from {row['guest_country']}, he booked a {row['accommodation_type']} in {row['accommodation_country']} for {row['room_nights']} nights during the month of {month}.
  The accommodation he booked had an everage grade of {row['accommodation_score']}/10 and a star rating of {row['accommodation_star_rating']} stars. The location of the accommodation was {row['location']}.
  """
  return text

In [11]:
tokenizer = AutoTokenizer.from_pretrained('bert-base-uncased')

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

In [10]:
import torch

# Check if GPU is available and set the device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [12]:
# def prepare_submission(user_accommodation_embeddings, review_embeddings, users_df, reviews_df):
#   submission = []

#   for idx, row in users_df.iterrows():
#       user_id = row['user_id']
#       accommodation_id = row['accommodation_id']

#       # Get the corresponding embedding
#       user_accommodation_embedding = user_accommodation_embeddings[idx]  # Shape: (embedding_dim,)

#       # Find all reviews for this accommodation
#       relevant_reviews = reviews_df[reviews_df['accommodation_id'] == accommodation_id]

#       if relevant_reviews.empty:
#           continue  # No reviews for this accommodation

#       # Encode only the relevant reviews
#       relevant_review_embeddings = review_embeddings[relevant_reviews.index]  # Shape: (num_reviews_for_accommodation, embedding_dim)

#       # Compute similarity scores
#       similarity_scores = (user_accommodation_embedding @ relevant_review_embeddings.T).cpu().numpy()

#       # Get top 10 most similar reviews
#       top_review_indices = similarity_scores.argsort()[-10:][::-1]  # Get top 10 indices

#       # Retrieve the corresponding review IDs
#       top_review_ids = relevant_reviews.iloc[top_review_indices]['review_id'].tolist()

#       # Ensure we have exactly 10 reviews (fill with -1 if not enough)
#       while len(top_review_ids) < 10:
#           top_review_ids.append(-1)  # Placeholder for missing reviews

#       # Store result
#       submission.append([accommodation_id, user_id] + top_review_ids)
#       submission_df = pd.DataFrame(submission, columns=['accommodation_id', 'user_id'] + [f'review_{i+1}' for i in range(10)])
#       return submission_df


In [12]:
import pandas as pd

def prepare_submission(user_accommodation_embeddings, review_embeddings, users_df, reviews_df):
    submission = []

    for idx, row in users_df.iterrows():
        user_id = row['user_id']
        accommodation_id = row['accommodation_id']

        # Get the corresponding embedding
        user_accommodation_embedding = user_accommodation_embeddings[idx]  # Shape: (embedding_dim,)

        # Find all reviews for this accommodation
        relevant_reviews = reviews_df[reviews_df['accommodation_id'] == accommodation_id]

        if relevant_reviews.empty:
            continue  # No reviews for this accommodation

        # Encode only the relevant reviews
        relevant_review_embeddings = review_embeddings[relevant_reviews.index]  # Shape: (num_reviews_for_accommodation, embedding_dim)

        # Compute similarity scores
        similarity_scores = (user_accommodation_embedding @ relevant_review_embeddings.T).detach().cpu().numpy()

        # Get top 10 most similar reviews
        top_review_indices = similarity_scores.argsort()[-10:][::-1]  # Get top 10 indices

        # Retrieve the corresponding review IDs
        top_review_ids = relevant_reviews.iloc[top_review_indices]['review_id'].tolist()

        # Ensure we have exactly 10 reviews (fill with -1 if not enough)
        while len(top_review_ids) < 10:
            top_review_ids.append(-1)  # Placeholder for missing reviews

        # Store result
        submission.append([accommodation_id, user_id] + top_review_ids)

    # **Move DataFrame creation & return OUTSIDE the loop**
    submission_df = pd.DataFrame(submission, columns=['accommodation_id', 'user_id'] + [f"review_{i+1}" for i in range(10)])
    return submission_df


In [21]:
accommodation_ids = users_df['accommodation_id'].unique()
len(accommodation_ids)

5000

In [31]:
filtered_acc_ids = accommodation_ids[0:0+1]

In [32]:
len(filtered_acc_ids)

1

In [33]:
reviews_indexed.loc[filtered_acc_ids]

,review_id,review_title,review_positive,review_negative,review_score,review_helpful_votes
accommodation_id,,,,,,
2086452554,21fa6e2e-4ea2-4310-9862-e2775dc9c6d8,NaN,Location - close to town,NaN,8.0,0
2086452554,4a0c680e-34ef-4076-aa47-bf843398ac35,NaN,"Nice and cozy, clean and comfortable room",NaN,9.0,0
2086452554,7fd07ac9-5ba2-41ad-982c-a2f81d7c8af0,NaN,"It was comfortable inside, &amp; had a little ...","We chose this place, as it was supposed to hav...",7.0,0
2086452554,29715049-e927-48a0-8c6a-f4fcd7de6087,NaN,Quiet setting yet close to town center Walk t...,NaN,9.0,0
2086452554,a250a1e9-67c4-47ef-8df2-9e0798c21ba7,We only stayed overnight but would have no qua...,Good location in a woodland setting within eas...,"Without wishing to be picky, the only things w...",9.0,0
...,...,...,...,...,...,...
2086452554,cac8dc3f-90f9-423a-9bdd-742bda022c4a,Would definitely stay again next time we are i...,"Beautiful location, very quiet. My partner and...",The water in the shower and sink takes a while...,10.0,0
2086452554,bef6e9c8-6684-4342-a90a-aabe33842034,NaN,Our host was great. Friendly and helpful with ...,NaN,10.0,0
2086452554,f844469c-516d-4962-a6cc-79ba9d0bef1e,Perfect for a one night stay,Location was amazing - walking distance to Chu...,NaN,9.0,0


In [13]:
start = 0
N = 10
accommodation_ids = users_df['accommodation_id'].unique()
counter = 1
users_indexed = users_df.set_index("accommodation_id")
reviews_indexed = reviews_df.set_index("accommodation_id")

for i in range(0, len(accommodation_ids), N):
  print(i)
  filtered_acc_ids = accommodation_ids[i:i+N]
  filtered_users_df = users_indexed.loc[filtered_acc_ids]
  filtered_users_df = filtered_users_df.reset_index()
  filtered_reviews_df = reviews_indexed.loc[filtered_acc_ids]
  filtered_reviews_df = filtered_reviews_df.reset_index()

  user_texts = filtered_users_df.apply(get_accommodation_text_representation, axis=1)
  user_tokens = tokenizer(user_texts.tolist(), padding=True, truncation=True, return_tensors='pt', max_length=64)
  user_tokens = {key: val.to(device) for key, val in user_tokens.items()}
  user_accommodation_embeddings = user_acc_encoder(user_tokens)

  review_texts = filtered_reviews_df .apply(get_review_text_representation_test, axis=1)
  review_tokens = tokenizer(review_texts.to_list(), padding=True, truncation=True, return_tensors='pt', max_length=64)
  review_tokens = {key: val.to(device) for key, val in review_tokens.items()}
  review_embeddings = review_encoder(review_tokens)

  submission_df = prepare_submission(user_accommodation_embeddings, review_embeddings, filtered_users_df, filtered_reviews_df)
  submission_df.to_csv(f"{PATH}/submission/submission_{counter}.csv", index=False)
  counter += 1

  del user_tokens, review_tokens, user_accommodation_embeddings, review_embeddings # Delete variables no longer needed
  torch.cuda.empty_cache() # Release unused GPU memory


0
10
20
30
40
50
60
70
80
90
100
110
120
130
140
150
160
170
180
190
200
210
220
230
240
250
260
270
280
290
300
310
320
330
340
350
360
370
380
390
400
410
420
430
440
450
460
470
480
490
500
510
520
530
540
550
560
570
580
590
600
610
620
630
640
650
660
670
680
690
700
710
720
730
740
750
760
770
780
790
800
810
820
830
840
850
860
870
880
890
900
910
920
930
940
950
960
970
980
990
1000
1010
1020
1030
1040
1050
1060
1070
1080
1090
1100
1110
1120
1130
1140
1150
1160
1170
1180
1190
1200
1210
1220
1230
1240
1250
1260
1270
1280
1290
1300
1310
1320
1330
1340
1350
1360
1370
1380
1390
1400
1410
1420
1430
1440
1450
1460
1470
1480
1490
1500
1510
1520
1530
1540
1550
1560
1570
1580
1590
1600
1610
1620
1630
1640
1650
1660
1670
1680
1690
1700
1710
1720
1730
1740
1750
1760
1770
1780
1790
1800
1810
1820
1830
1840
1850
1860
1870
1880
1890
1900
1910
1920
1930
1940
1950
1960
1970
1980
1990
2000
2010
2020
2030
2040
2050
2060
2070
2080
2090
2100
2110
2120
2130
2140
2150
2160
2170
2180
2190
2200
2210
2

In [17]:
user_accommodation_embeddings.shape

torch.Size([996, 128])

In [19]:
del user_tokens, review_tokens, user_accommodation_embeddings, review_embeddings # Delete variables no longer needed

In [2]:
# # Encode (user, accommodation) pairs
# user_texts = users_df.apply(get_accommodation_text_representation, axis=1)
# user_tokens = tokenizer(user_texts.tolist(), padding=True, truncation=True, return_tensors='pt', max_length=64)

# # Move user_tokens to the device
# user_tokens = {key: val.to(device) for key, val in user_tokens.items()}

# user_accommodation_embeddings = user_acc_encoder(user_tokens)  # Shape: (num_user_accommodations, embedding_dim)

# # Encode all reviews
# review_texts = reviews_df.apply(get_review_text_representation_test, axis=1)
# review_tokens = tokenizer(review_texts.to_list(), padding=True, truncation=True, return_tensors='pt', max_length=64)

# # Move review_tokens to the device
# review_tokens = {key: val.to(device) for key, val in review_tokens.items()}

# review_embeddings = review_encoder(review_tokens)  # Shape: (num_reviews, embedding_dim)

In [14]:
import pandas as pd
import os

folder_path = f"{PATH}/submission/"  # Change this to your actual folder path
output_file = f"{PATH}/final_submission.csv"  # Change this to your desired output file

# Initialize an empty list to store DataFrames
dfs = []

# Loop through submission_1.csv to submission_500.csv
for i in range(1, 501):
    file_path = os.path.join(folder_path, f"submission_{i}.csv")
    if os.path.exists(file_path):  # Ensure file exists before reading
        dfs.append(pd.read_csv(file_path))

# Concatenate all DataFrames
df_final = pd.concat(dfs, ignore_index=True)

# Save the concatenated DataFrame
df_final.to_csv(output_file, index=False)

print(f"Concatenated {len(dfs)} files into {output_file}")


Concatenated 500 files into /content/drive/MyDrive/kaggle_competition_data//final_submission.csv


In [15]:
submission_df = pd.read_csv(f"{PATH}/final_submission.csv")
if 'ID' not in submission_df.columns:
    submission_df.insert(0, 'ID', range(1, len(submission_df) + 1))

submission_df.to_csv('submission_formatted.csv', index=False)


In [16]:
len(submission_df)

199138

In [17]:
submission_df.to_csv(f"{PATH}/submission_formatted.csv", index=False)

In [27]:
import os
import glob
  # Change this to your target folder

for file in glob.glob(os.path.join(f"{PATH}/submission", "submission*")):
    os.remove(file)

In [20]:
len(submission_df)

500